In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv(r"E:\Deep learning UDEMY\P16-Boltzmann-Machines\Boltzmann_Machines\ml-1m\movies.dat", sep= "::", header= None, engine= "python", encoding= "latin-1")
users = pd.read_csv(r"E:\Deep learning UDEMY\P16-Boltzmann-Machines\Boltzmann_Machines\ml-1m\users.dat", sep= "::", header= None, engine= "python", encoding= "latin-1")
ratings = pd.read_csv(r"E:\Deep learning UDEMY\P16-Boltzmann-Machines\Boltzmann_Machines\ml-1m\ratings.dat", sep= "::", header= None, engine= "python", encoding= "latin-1")

In [3]:
trainig_set = pd.read_csv(r"E:\Deep learning UDEMY\P16-Boltzmann-Machines\Boltzmann_Machines\ml-100k\u1.base", delimiter="\t", )
trainig_set = np.array(trainig_set, dtype = "int")

In [4]:
test_set = pd.read_csv(r"E:\Deep learning UDEMY\P16-Boltzmann-Machines\Boltzmann_Machines\ml-100k\u1.test", delimiter="\t", )
test_set = np.array(test_set, dtype = "int")

In [7]:
def movies(train_set, test_set):
    total_movies = 0
    for i in train_set[:, 1]:
        if i > total_movies:
            total_movies = i

    for i in test_set[:, 1]:
        if i > total_movies:
            total_movies = i

    return total_movies

def users(train_set, test_set):
    total_users = 0
    for i in train_set[:, 0]:
        if i > total_users:
            total_users = i

    for i in test_set[:, 0]:
        if i > total_users:
            total_users = i

    return total_users

In [8]:
total_num_movies = movies(trainig_set, test_set)
total_num_users = users(trainig_set, test_set)
print(total_num_movies)
print(total_num_users)

1682
943


In [9]:
def converter(set):
    li = []
    for i in range(1, total_num_users + 1):
        id_movies = (set[:, 1] [set[:, 0] == i])
        id_rating = (set[:, 2] [set[:, 0] == i])
        rate = np.zeros(total_num_movies)
        rate[id_movies - 1] = id_rating
        li.append(list(rate))
    return li

In [10]:
trainig_set = converter(trainig_set)
test_set = converter(test_set)

In [11]:
torch_train = torch.FloatTensor(trainig_set)
torch_test = torch.FloatTensor(test_set)

In [12]:
def binary_converter(data):
    data[data == 0] = -1
    data[data == 1] = 0
    data[data == 2] = 0
    data[data >= 3] = 1
    return data

In [13]:
torch_train = binary_converter(torch_train)
torch_test = binary_converter(torch_test)

In [14]:
torch_train

tensor([[-1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.]])

In [16]:
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
nv = len(torch_train[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [18]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, total_num_users - batch_size, batch_size):
    vk = torch_train[id_user : id_user + batch_size]
    v0 = torch_train[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3560)
epoch: 2 loss: tensor(0.2440)
epoch: 3 loss: tensor(0.2501)
epoch: 4 loss: tensor(0.2465)
epoch: 5 loss: tensor(0.2500)
epoch: 6 loss: tensor(0.2523)
epoch: 7 loss: tensor(0.2491)
epoch: 8 loss: tensor(0.2496)
epoch: 9 loss: tensor(0.2454)
epoch: 10 loss: tensor(0.2490)


In [19]:
test_loss = 0
s = 0.
for id_user in range(total_num_users):
    v = torch_train[id_user:id_user+1]
    vt = torch_test[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2474)
